# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

# Naive Bayes 

In [33]:
%pip install -r ../requirements.txt


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
!pip install scikit-learn


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [35]:
!pip install nltk


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [36]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import pickle
import nltk
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.naive_bayes import GaussianNB


In [37]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
# Read csv

df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv')

In [39]:
# mirando el dataset:
df_raw.head(10)

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0
5,com.facebook.katana,idk i can't edit my posts? things such as my ...,0
6,com.facebook.katana,major flaws constant updates and always getti...,0
7,com.facebook.katana,video issues since i was forced into this upd...,0
8,com.facebook.katana,this update completely destroyed my facebook...,0
9,com.facebook.katana,"posting issues for the last week, there's bee...",0


In [40]:
# viendo información del dataset
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [41]:
# hacemos un sample:
df_raw.sample(5)

,package_name,review,polarity
599,com.evernote,"absolutely the best for saving documents, fo...",1
60,com.twitter.android,awsome great type of social media nothing wr...,0
396,com.facebook.orca,my husband and i can't send msgs to each oth...,0
119,com.king.candycrushsaga,why do u do this to me ok so i can't unlock t...,0
822,com.hamropatro,it's really nice app.. i love this app.,1


In [42]:
# contamos la variable a predecir:
# 0 negativo 
# 1 positivo
df_raw['polarity'].value_counts()

0    584
1    307
Name: polarity, dtype: int64

***Transformar el Dataset***

In [43]:
df_transf = df_raw.copy()

In [44]:
# Drop package_name column

df_transf = df_transf.drop('package_name', axis=1)

In [45]:
# miramos la transformación que hicimos:
df_transf.head(1)

,review,polarity
0,privacy at least put some option appear offli...,0


In [46]:
# trabajamos sobre la columna review
df_transf['review'] = df_transf['review'].str.strip()
# elimina espacio libre al principio y al final

In [47]:
# column review to lower case
df_transf['review'] = df_transf['review'].str.lower()

In [48]:
stop = stopwords.words('english')

def clean_words(review):
    if review is not None:
        words = review.strip().split()
        new_words = []
        for word in words:
            if word not in stop:new_words.append(word)
        Result = ' '.join(new_words)    
    else:
        Result = None
    return Result


In [49]:
df_transf['review'] = df_transf['review'].apply(clean_words)

In [50]:
df_transf['review'].str.split(expand=True).stack().value_counts()[:60]

app         317
can't       137
new         136
update      136
like        135
fix         133
please      131
good        128
game        121
one         119
use         118
get         112
even        101
great        99
time         93
really       82
would        80
back         78
still        78
love         77
browser      76
make         74
see          69
using        68
work         67
i'm          67
5            67
open         65
want         64
it.          64
google       63
much         61
phone        61
every        61
u            58
option       58
version      57
go           56
way          56
app.         56
used         55
also         55
android      54
download     53
best         52
.            51
give         50
better       50
play         50
send         49
people       48
able         47
message      47
since        46
many         45
keep         45
works        44
i've         44
old          43
find         43
dtype: int64

In [51]:
# llamamos a la transf.copy = df
df = df_transf.copy()

In [52]:
# ver los datos de df 
df.head(5)

,review,polarity
0,privacy least put option appear offline. mean ...,0
1,"messenger issues ever since last update, initi...",0
2,profile time wife anybody one post view would ...,0
3,new features suck us working back button guys ...,0
4,forced reload uploading pic replying comment l...,0


**Split data frame**

In [53]:
X = df['review']
y = df['polarity']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=25)


In [54]:
# mirando datos:
print(X_train.shape)

(712,)


In [55]:
# mirando datos:
print(X_test.shape)

(179,)


In [56]:
# mirando datos:
y_train.value_counts()

0    467
1    245
Name: polarity, dtype: int64

In [57]:
# mirando datos:
y_test.value_counts()

0    117
1     62
Name: polarity, dtype: int64

**Pipeline with two pre-processing steps and one modeling step**

In [58]:
# Vectorize text reviews to numbers: 

vect = CountVectorizer() # vector de conteo

text_vec = vect.fit_transform(X_train)

In [59]:
vect.get_feature_names_out() #da nombres de las columnas

array(['04', '0x', '10', ..., 'žŕľ', 'ˇŕ', 'ˇŕľ'], dtype=object)

In [60]:
vect_tfidf = TfidfVectorizer()

text_vec_tfidf = vect_tfidf.fit_transform(X_train)

In [61]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer())])
text_norm = text_clf.fit_transform(X_train)

In [62]:
clf_1 = MultinomialNB()

clf_1.fit(text_vec, y_train)

MultinomialNB()

In [63]:
clf_2 = MultinomialNB()

clf_2.fit(text_vec_tfidf, y_train)

MultinomialNB()

In [64]:
clf_3 = MultinomialNB()

clf_3.fit(text_norm, y_train)

MultinomialNB()

In [65]:
pred_1 = clf_1.predict(vect.transform(X_test))
pred_2 = clf_2.predict(vect_tfidf.transform(X_test))
pred_3 = clf_3.predict(text_clf.transform(X_test))

print(classification_report(y_test, pred_1))
print(classification_report(y_test, pred_2))
print(classification_report(y_test, pred_3))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       117
           1       0.86      0.61      0.72        62

    accuracy                           0.83       179
   macro avg       0.84      0.78      0.80       179
weighted avg       0.84      0.83      0.82       179

              precision    recall  f1-score   support

           0       0.70      0.99      0.82       117
           1       0.93      0.21      0.34        62

    accuracy                           0.72       179
   macro avg       0.82      0.60      0.58       179
weighted avg       0.78      0.72      0.66       179

              precision    recall  f1-score   support

           0       0.70      0.99      0.82       117
           1       0.93      0.21      0.34        62

    accuracy                           0.72       179
   macro avg       0.82      0.60      0.58       179
weighted avg       0.78      0.72      0.66       179



In [67]:
# Acá todo junto en un pipeline

text_clf_2 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf_2.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

**Check Result**

In [70]:
y_pred = text_clf_2.predict(X_test)
precision_recall_fscore_support(y_test, y_pred, average='weighted')

(0.7811506928825365, 0.7206703910614525, 0.6562337735459534, None)

In [71]:
print('Naive Bayes Train Accuracy = ',metrics.accuracy_score(y_train,text_clf_2.predict(X_train)))
print('Naive Bayes Test Accuracy = ',metrics.accuracy_score(y_test,text_clf_2.predict(X_test)))

Naive Bayes Train Accuracy =  0.8693820224719101
Naive Bayes Test Accuracy =  0.7206703910614525


**Randomized search to select hyperparameters**

In [72]:
n_iter_search = 5
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = RandomizedSearchCV(text_clf_2, parameters, n_iter = n_iter_search)
gs_clf.fit(X_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('tfidf', TfidfTransformer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=5,
                   param_distributions={'clf__alpha': (0.01, 0.001),
                                        'tfidf__use_idf': (True, False),
                                        'vect__ngram_range': [(1, 1), (1, 2)]})

In [73]:
gs_clf.best_params_

{'vect__ngram_range': (1, 1), 'tfidf__use_idf': False, 'clf__alpha': 0.01}

In [74]:
print('Naive Bayes Train Accuracy (grid random search) = ',metrics.accuracy_score(y_train,gs_clf.predict(X_train)))
print('Naive Bayes Test Accuracy (grid random search) = ',metrics.accuracy_score(y_test,gs_clf.predict(X_test)))

Naive Bayes Train Accuracy (grid random search) =  0.9887640449438202
Naive Bayes Test Accuracy (grid random search) =  0.8268156424581006


In [75]:
text_clf_count_vect = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])
text_clf_count_vect.fit(X_train, y_train)


n_iter_search = 5
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'clf__alpha': (1e-2, 1e-3)}
gs_count_vect = RandomizedSearchCV(text_clf_count_vect, parameters, n_iter = n_iter_search)
gs_count_vect.fit(X_train, y_train)

/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=5. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=5,
                   param_distributions={'clf__alpha': (0.01, 0.001),
                                        'vect__ngram_range': [(1, 1), (1, 2)]})

In [76]:
gs_count_vect.best_params_

{'vect__ngram_range': (1, 1), 'clf__alpha': 0.01}

In [77]:
print('Naive Bayes Train Accuracy (grid random search) = ',metrics.accuracy_score(y_train,gs_count_vect.predict(X_train)))
print('Naive Bayes Test Accuracy (grid random search) = ',metrics.accuracy_score(y_test,gs_count_vect.predict(X_test)))

Naive Bayes Train Accuracy (grid random search) =  0.9915730337078652
Naive Bayes Test Accuracy (grid random search) =  0.8379888268156425


In [78]:
y_pred_mejor = gs_clf.predict(X_test)

print(classification_report(y_test, y_pred_mejor))

              precision    recall  f1-score   support

           0       0.81      0.96      0.88       117
           1       0.88      0.58      0.70        62

    accuracy                           0.83       179
   macro avg       0.84      0.77      0.79       179
weighted avg       0.83      0.83      0.82       179



In [79]:
best_model = gs_clf.best_estimator_
best_model

Pipeline(steps=[('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(alpha=0.01))])

In [80]:
# Save best model

pickle.dump(best_model, open('../models/best_model.pickle', 'wb'))

In [81]:
modelo = pickle.load(open('../models/best_model.pickle', 'rb')) # lo leemos
modelo.predict(X_test) # lo usamos para predecir nueva X_test

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0])